In [2]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
import tensorflow
import tensorflow as tf
import random
import json
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Shivam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
with open('intents.json') as f:
    data = json.load(f)

{'intents': [{'tag': 'Cuts',
   'patterns': ['What to do if Cuts?',
    'How to cure Cuts?',
    'Which medicine to apply for Cuts?',
    'what to apply on cuts?',
    'Cuts'],
   'responses': ['Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile bandage. Pain relievers such as acetaminophen can be applied.'],
   'context_set': ''},
  {'tag': 'Abrasions',
   'patterns': ['how do you treat abrasions?',
    'Do Abrasions cause scars?',
    'Abrasions',
    'what to do if abrasions?',
    'Which medicine to apply for abrasions?',
    'How to cure abrasions?'],
   'responses': ['Begin with washed hands.Gently clean the area with cool to lukewarm water and mild soap. Remove dirt or other particles from the wound using sterilized tweezers.For a mild scrape that’s not bleeding, leave the wound uncovered.If the

In [23]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
  for pattern in intent["patterns"]:
    wrds = nltk.word_tokenize(pattern)
    words.extend(wrds)
    docs_x.append(wrds)
    docs_y.append(intent["tag"])


  if intent["tag"] not in labels:
    labels.append(intent["tag"])  

words = [stemmer.stem(w.lower()) for w in words if  w not in "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x,doc in enumerate(docs_x):
  bag = []
  wrds = [stemmer.stem(w) for w in doc]

  for w in words:
    if w in wrds:
      bag.append(1)
    else:
      bag.append(0)

  output_row = out_empty[:]
  output_row[labels.index(docs_y[x])] = 1

  training.append(bag)
  output.append(output_row)


In [29]:

training = np.array(training)
output = np.array(output)

In [32]:
training.shape

(188, 114)

In [33]:
output.shape

(188, 44)

In [34]:
model = Sequential()
model.add(Dense(8, input_shape=(len(training[0]),), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(len(output[0]), activation='softmax'))


In [35]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [36]:
model.fit(training, output, epochs=50, batch_size=8)

Train on 188 samples
Epoch 1/50
188/188 [==============================] - 0s 402us/sample - loss: 3.7859 - acc: 0.0106
Epoch 2/50
188/188 [==============================] - 0s 107us/sample - loss: 3.7749 - acc: 0.0160
Epoch 3/50
188/188 [==============================] - 0s 158us/sample - loss: 3.7661 - acc: 0.0213
Epoch 4/50
188/188 [==============================] - 0s 140us/sample - loss: 3.7574 - acc: 0.0213
Epoch 5/50
188/188 [==============================] - 0s 109us/sample - loss: 3.7483 - acc: 0.0266
Epoch 6/50
188/188 [==============================] - 0s 112us/sample - loss: 3.7367 - acc: 0.0266
Epoch 7/50
188/188 [==============================] - 0s 157us/sample - loss: 3.7230 - acc: 0.0266
Epoch 8/50
188/188 [==============================] - 0s 149us/sample - loss: 3.7042 - acc: 0.0319
Epoch 9/50
188/188 [==============================] - 0s 122us/sample - loss: 3.6825 - acc: 0.0266
Epoch 10/50
188/188 [==============================] - 0s 146us/sample - loss: 3.6560 - 

In [37]:
def bag_of_words(s, words):
  bag = [0 for _ in range(len(words))]

  s_words = nltk.word_tokenize(s)
  s_words = [stemmer.stem(word.lower()) for word in s_words]

  for se in s_words:
    for i, w in enumerate(words):
      if w == se:
        bag[i] = 1

  return np.array(bag)

In [46]:
training[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0])

In [44]:
bag_of_words("How to cure fever", words)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0])

(188, 114)
Train on 188 samples
Epoch 1/50
188/188 [==============================] - 0s 451us/sample - loss: 3.7791 - acc: 0.0213
Epoch 2/50
188/188 [==============================] - 0s 89us/sample - loss: 3.7660 - acc: 0.0319
Epoch 3/50
188/188 [==============================] - 0s 93us/sample - loss: 3.7543 - acc: 0.0532
Epoch 4/50
188/188 [==============================] - 0s 96us/sample - loss: 3.7419 - acc: 0.0638
Epoch 5/50
188/188 [==============================] - 0s 103us/sample - loss: 3.7262 - acc: 0.0691
Epoch 6/50
188/188 [==============================] - 0s 100us/sample - loss: 3.7075 - acc: 0.0691
Epoch 7/50
188/188 [==============================] - 0s 95us/sample - loss: 3.6823 - acc: 0.0851
Epoch 8/50
188/188 [==============================] - 0s 106us/sample - loss: 3.6531 - acc: 0.0851
Epoch 9/50
188/188 [==============================] - 0s 98us/sample - loss: 3.6168 - acc: 0.0798
Epoch 10/50
188/188 [==============================] - 0s 102us/sample - loss: 3.5

c:\Users\Shivam\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [56]:
# results = model.predict(training[0].reshape(1, -1))
# results_index = np.argmax(results)
# tag = labels[results_index]
# print(tag)


(1, 114)

In [57]:


def chat():
  print("\033[1;31m Start talking with the bot!, Type quit when you want to quit")
  while True:
    inp = input("\033[You: ")
    if inp.lower() == "quit":
      break

    print(inp)

    results = model.predict([bag_of_words(inp, words).reshape(1, -1)])
    results_index = np.argmax(results)
    tag = labels[results_index]

    for tg in data["intents"]:
      
      if tg['tag'] == tag:
        
        responses = tg['responses']

        response_list = nltk.sent_tokenize(str(responses[0]))
        colors = [31, 32, 33, 34, 35]

        for i in range(len(response_list)):
            color_index = i % len(colors)
            print(f'\033[1;{colors[color_index]}m {response_list[i]:<12s}\n')


In [58]:
chat()

 Start talking with the bot!, Type quit when you want to quit
How to cure cuts
 Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops.

 Apply Petroleum Jelly to make sure that the wound is moist for quick healing.

 Finally cover the cut with a sterile bandage.

 Pain relievers such as acetaminophen can be applied.

howt to cure cuts?
 Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops.

 Apply Petroleum Jelly to make sure that the wound is moist for quick healing.

 Finally cover the cut with a sterile bandage.

 Pain relievers such as acetaminophen can be applied.

